In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
import random

<h1> get data from database</h1>

In [6]:
import mysql.connector
from mysql.connector import Error
records=[]
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='optimatets',
                                         user='root',
                                         password='')
    sql_select_Query=[]
    sql_select_Query.append( "select * from bookings")
    sql_select_Query.append("select * from driver_vehicle")
    sql_select_Query.append("select * from tracking")
    sql_select_Query.append("select * from vehicles")
    sql_select_Query.append("select * from vehicle_types")
    for req in sql_select_Query:
        cursor = connection.cursor()
        cursor.execute(req)
        records.append( cursor.fetchall())    

except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
        print("MySQL connection is closed")

MySQL connection is closed


In [7]:
bookings=pd.DataFrame(records[0],columns=['id', 'book_type', 'promo_id' , 'type_id' , 'accept_status', 'total_weight', 'company_id', 'date_time', 'user_id', 'vehicle_id', 'driver_id', 'customer_id', 'pickup', 'dropoff', 'duration', 'pickup_addr', 'dest_addr','note', 'travellers', 'status', 'payment', 'created_at', 'updated_at', 'deleted_at'])


In [8]:
df1=bookings


In [25]:
df1

,id,book_type,promo_id,type_id,accept_status,total_weight,company_id,date_time,user_id,vehicle_id,...,duration,pickup_addr,dest_addr,note,travellers,status,payment,created_at,updated_at,deleted_at
0,4,s,NaN,17,53,30.0,11.0,2019-07-03 12:01:13,53,21.0,...,None,TUNIS,SOUSSE,None,1,0,0,2019-07-01 11:02:24,2019-07-29 12:20:08,2019-07-29 12:20:08
1,8,s,NaN,17,53,20.0,11.0,2019-07-06 07:09:11,53,21.0,...,None,SFAX,TUNIS,None,1,0,0,2019-07-04 14:13:04,2019-07-29 12:20:16,2019-07-29 12:20:16
2,9,s,NaN,6,60,10.0,11.0,2019-07-08 08:21:49,60,7.0,...,None,TUNIS,BIZERTE,None,1,0,0,2019-07-04 14:23:12,2019-07-29 12:20:25,2019-07-29 12:20:25
3,14,s,NaN,3,51,5.0,10.0,2019-07-11 09:06:34,51,3.0,...,None,TUNIS,SOUSSE,Bien reçu. Je note çà appele moi sur 22222222,1,3,1,2019-07-09 11:08:28,2019-07-29 12:21:40,2019-07-29 12:21:40
4,15,s,NaN,17,60,25.0,11.0,2019-07-14 12:24:05,60,7.0,...,None,TUNIS,MAHDIA,None,1,1,0,2019-07-09 11:24:51,2019-07-29 12:21:47,2019-07-29 12:21:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,252,s,NaN,7,0,21.0,NaN,2020-02-10 09:47:01,66,8.0,...,None,ARIANA,SFAX,None,1,0,0,2020-02-07 09:46:38,2020-02-07 09:46:38,NaT
193,253,s,NaN,2,0,5.0,NaN,2020-02-08 10:21:46,66,19.0,...,None,SFAX,TUNIS,None,1,0,0,2020-02-07 10:21:36,2020-02-07 10:21:36,NaT
194,254,s,NaN,2,0,5.0,NaN,2020-02-08 10:21:46,66,10.0,...,None,SFAX,TUNIS,None,1,0,0,2020-02-07 10:21:43,2020-02-07 10:21:43,NaT
195,255,p,98.0,1,65,2.0,13.0,2020-02-08 08:34:29,65,52.0,...,None,TUNIS,SFAX,None,1,3,0,2020-02-07 15:35:24,2020-02-07 15:45:54,NaT


<h1> adress data normalisation <h1>

In [9]:

region=['Tunis','Manouba','ben arous',
        'ariana',' Bizerte','beja', 'jandouba', 'nabeul' , 'zaghouane', 'silana', 'kef' ,
        'kasserine' , 'kairouane' , 'Sousse' , 'monastir', 'Mahdia' , 'Sfax' , 'sidi' 'bouzid' , 'gafsa' ,
        'touzeur', 'kbeli' , 'gabes' , 'mednine' ,'tataouine']
region=[x.upper() for x in region]
destination_adress= df1['dest_addr'] 
depart_adress = df1['pickup_addr']

In [10]:

def adressNormalisation(col):
    i=0
    lista=[]
    adr=''
    for phrase in col:
        verif=False
        for adrs in region:
            if adrs in phrase.upper():
                verif=True
                adr=adrs
        if verif==True :
                #lista.append([i,phrase,adr])
                lista.append([adr])

        else : 
                #lista.append([i,phrase,random.choice(region)])
                lista.append([random.choice(region)])


        i=i+1


    return pd.DataFrame(lista, columns=['new_'+col.name])



In [11]:

def adressToNan(col):
    i=0
    lista=[]
    adr=''
    for phrase in col:
        verif=False
        for adrs in region:
            if adrs in phrase.upper():
                verif=True
                adr=adrs
        if verif==True :
                #lista.append([i,phrase,adr])
                lista.append([adr])

        else : 
                #lista.append([i,phrase,random.choice(region)])
                lista.append([np.nan])


        i=i+1


    return pd.DataFrame(lista, columns=['new_'+col.name])



In [12]:
df1['pickup_addr']=adressNormalisation(df1['pickup_addr'])
df1['dest_addr']=adressNormalisation(df1['dest_addr'])

<h1> fill the nan vehicle_id </h1>
<h4>this method return a random vihicle_id by adress </h4>

In [13]:
df2=df1
def getVihicleByAdress(df2, adress):
    df2=df2.dropna(subset=["vehicle_id"])
    df2=df2.groupby('pickup_addr')['vehicle_id'].apply(list)
    x=list(set(df2[adress]))
    return random.choice(x)
 



In [14]:
df2.groupby('pickup_addr')['vehicle_id'].apply(list)

pickup_addr
ARIANA                                  [8.0, 8.0, nan, 8.0]
BEJA                                            [23.0, 23.0]
GABES              [nan, nan, nan, nan, 21.0, nan, nan, nan]
MANOUBA                                                [5.0]
SFAX       [nan, 9.0, nan, nan, nan, 10.0, nan, 10.0, 9.0...
SOUSSE     [nan, nan, nan, 25.0, 21.0, nan, 21.0, 22.0, n...
TUNIS      [nan, 7.0, 3.0, 7.0, nan, nan, nan, 13.0, nan,...
Name: vehicle_id, dtype: object

In [15]:
for i in range(len(df1['vehicle_id'])):
    if np.isnan(df1['vehicle_id'].iloc[i]):
        df1['vehicle_id'].iloc[i]=getVihicleByAdress(df1,df1['pickup_addr'].iloc[i])

C:\Users\meher\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


<h1> prepare train and test data </h1>

In [16]:
vihicule_id=df1['vehicle_id'].unique()


In [17]:
df = pd.DataFrame()
df['Datetime'] = df1['date_time']
for i in range(len(df1)):
    
    df['Year'] = df.Datetime.dt.year
    df['Month'] = df.Datetime.dt.month
    df['Day'] = df.Datetime.dt.day
    df['Hour'] = df.Datetime.dt.hour
    #df['Minute'] = df.Datetime.dt.minute


In [18]:
y=df1[['dest_addr','pickup_addr']]

In [19]:
x=pd.concat([df[['Year','Day','Month','Hour']],df1['vehicle_id']],sort=False,axis=1)


In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)


<h1>train model</h1>

In [21]:

# Compare Algorithms

import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
# load dataset

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
scores = []
scoring = 'accuracy'

for name, model in models:
  
    cv_results = MultiOutputClassifier(model).fit(x_train,y_train)
   
    results.append(cv_results)
    names.append(name)
    msg = "the score of %s  is : %s " % (name, cv_results.score(x_test,y_test))
    scores.append(cv_results.score(x_test,y_test))
    print(msg)
# boxplot algorithm comparison



the score of LR  is : 0.26666666666666666 
the score of LDA  is : 0.16666666666666666 
the score of KNN  is : 0.4 
the score of CART  is : 0.43333333333333335 
the score of NB  is : 0.13333333333333333 
the score of SVM  is : 0.06666666666666667 


C:\Users\meher\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\meher\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [22]:
def get_model(model):
    models = {
        'LR': 0,
        'LDA': 1,
        'KNN': 2,
        "DT": 3,
        'GNB': 4,
        'SVM': 5,
    }
    return (models.get(model, "Invalid model")) 

In [23]:

import mysql.connector
from mysql.connector import Error

def get_vehicle(ids,model_name):

    try:
        connection = mysql.connector.connect(host='localhost',
                                         database='optimatets',
                                         user='root',
                                         password='')
        records=[]  
        sql_select_Query=[]
        vihicle=[]
        for i in ids:        
            reqet = "SELECT department_id,company_id,make,model,type_id FROM vehicles WHERE id='" + str(i) + "';"
            sql_select_Query.append(reqet)
       
        for req in sql_select_Query:
            cursor = connection.cursor()
            cursor.execute(req)
            ex=cursor.fetchall()[0]
            msg="this vihicle of the company : "+str(ex[1])+" , departement : "+str(ex[0]) +" with the make : "+ ex[2]+" and a model : "+ex[3]
            records.append(ex)    
            vihicle.append({'message':msg,'type_id':ex[4],"score":scores[get_model(model_name)]})
    except Error as e:
        print("Error reading data from MySQL table", e)
    finally:
        if (connection.is_connected()):
            connection.close()
            cursor.close()
            print("MySQL connection is closed")
    
    return(vihicle)
      

In [24]:
from flask import Flask, request, make_response
import json
app = Flask(__name__)
@app.route('/', methods=['POST'])
def index():
    
    d = request.form.to_dict() # data recived from form
    date=d['date'].split("T")
    date1=date[0].split("-")
    hours=date[1].split(':')
    date2=list(map(int, date1)) 
    hours2= list(map(int, hours)) 
   
    
    df=pd.DataFrame(columns=['Datatime'],data=np.array(range(len(vihicule_id))))
   
    for i in range(len(vihicule_id)):  
    
        df['Year'] =date1[0]
        df['Month'] =date1[1]
        df['Day'] = date1[2]
        df['Hour'] =int(hours[0])
        
        
    

    xx=pd.concat([df[['Year','Day','Month','Hour']],pd.Series(vihicule_id).rename("id")],sort=False,axis=1) #input data
    result=pd.concat([pd.DataFrame(results[get_model(d['model'])].predict(np.array(xx))),xx],sort=False,axis=1)                 #result of prediction
    result=result.rename(columns={0: 'destination',1:'pickup'})
    ms=result[(result.destination == str( d['destination']).upper()) & (result.pickup == str(d['retour']).upper())] # return only the result of our destination and pickup
        
    if ms.empty:
        vehicles=[{'message': 'there is no vehicles , please change the time or pickup/destination','type_id': -1,'score':scores[get_model(d['model'])]}]
    else:
        
        vehicles= get_vehicle(ms['id'].tolist(),d['model'])       # return the data of vehicles predicted                                                
    
    
   
    
   
    
    print(d)        # data recived from form
    print(result)    # result of the prediction of all vehicles
    print(ms)        # result of the prediction of  vehicle who strat of our pickup and go to destination
    print(vehicles) # the data of vehicle predicted
    
    
    data=make_response(json.dumps(vehicles)) # transform final result to json
    resp=data
    resp.status_code = 200
    resp.headers['Access-Control-Allow-Origin'] = '*' # configiration 
    return resp


    
    
    
if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Sep/2020 08:51:39] "POST / HTTP/1.1" 200 -


{'date': '2020-12-11T08:15', 'retour': 'tunis', 'destination': 'sousse', 'model': 'KNN'}
   destination  pickup  Year Day Month  Hour    id
0         SFAX  SOUSSE  2020  11    12     8  21.0
1        TUNIS   TUNIS  2020  11    12     8   7.0
2        TUNIS   TUNIS  2020  11    12     8   3.0
3        TUNIS    SFAX  2020  11    12     8   9.0
4        TUNIS    SFAX  2020  11    12     8  10.0
5        TUNIS    SFAX  2020  11    12     8  11.0
6         SFAX  SOUSSE  2020  11    12     8  16.0
7        TUNIS    SFAX  2020  11    12     8  13.0
8         SFAX   TUNIS  2020  11    12     8  52.0
9         SFAX  SOUSSE  2020  11    12     8  18.0
10        SFAX   TUNIS  2020  11    12     8  51.0
11        SFAX  SOUSSE  2020  11    12     8  19.0
12        SFAX  SOUSSE  2020  11    12     8  17.0
13        SFAX   TUNIS  2020  11    12     8  46.0
14       TUNIS   TUNIS  2020  11    12     8   5.0
15        SFAX  SOUSSE  2020  11    12     8  23.0
16        SFAX  SOUSSE  2020  11    12     8

127.0.0.1 - - [14/Sep/2020 08:52:19] "POST / HTTP/1.1" 200 -


MySQL connection is closed
{'date': '2020-12-11T08:15', 'retour': 'tunis', 'destination': 'sfax', 'model': 'KNN'}
   destination  pickup  Year Day Month  Hour    id
0         SFAX  SOUSSE  2020  11    12     8  21.0
1        TUNIS   TUNIS  2020  11    12     8   7.0
2        TUNIS   TUNIS  2020  11    12     8   3.0
3        TUNIS    SFAX  2020  11    12     8   9.0
4        TUNIS    SFAX  2020  11    12     8  10.0
5        TUNIS    SFAX  2020  11    12     8  11.0
6         SFAX  SOUSSE  2020  11    12     8  16.0
7        TUNIS    SFAX  2020  11    12     8  13.0
8         SFAX   TUNIS  2020  11    12     8  52.0
9         SFAX  SOUSSE  2020  11    12     8  18.0
10        SFAX   TUNIS  2020  11    12     8  51.0
11        SFAX  SOUSSE  2020  11    12     8  19.0
12        SFAX  SOUSSE  2020  11    12     8  17.0
13        SFAX   TUNIS  2020  11    12     8  46.0
14       TUNIS   TUNIS  2020  11    12     8   5.0
15        SFAX  SOUSSE  2020  11    12     8  23.0
16        SFAX  SOU

[2020-09-14 08:53:06,952] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-24-9bee8f2a70a1>", line 8, in index
    date=d['date'].split("T")
